<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#Приведение-текстов-к-нижнему-регистру" data-toc-modified-id="Приведение-текстов-к-нижнему-регистру-2.0.0.2"><span class="toc-item-num">2.0.0.2&nbsp;&nbsp;</span>Приведение текстов к нижнему регистру</a></span></li><li><span><a href="#Замена-англицизмов" data-toc-modified-id="Замена-англицизмов-2.0.0.3"><span class="toc-item-num">2.0.0.3&nbsp;&nbsp;</span>Замена англицизмов</a></span></li><li><span><a href="#Замена-числительных" data-toc-modified-id="Замена-числительных-2.0.0.4"><span class="toc-item-num">2.0.0.4&nbsp;&nbsp;</span>Замена числительных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.0.0.5"><span class="toc-item-num">2.0.0.5&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Подготовка-корпуса" data-toc-modified-id="Подготовка-корпуса-2.0.0.6"><span class="toc-item-num">2.0.0.6&nbsp;&nbsp;</span>Подготовка корпуса</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Тематическое-моделирвование" data-toc-modified-id="Тематическое-моделирвование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тематическое моделирвование</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#LDA" data-toc-modified-id="LDA-3.0.0.1"><span class="toc-item-num">3.0.0.1&nbsp;&nbsp;</span>LDA</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Импорт библиотек

In [1]:
import pandas as pd
import re
import json
import glob
from sklearn.feature_extraction.text import CountVectorizer

#pickle
import pickle

#spacy
import spacy
import nltk
from nltk.corpus import stopwords 

import numpy as np
from pymystem3 import Mystem
m = Mystem() 
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.express as px
import gensim.corpora as corpora
import gensim.models


#Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.utils import tokenize
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

#vis
import pyLDAvis
import pyLDAvis.gensim_models
import plotly
import plotly.graph_objs as go
import plotly.express as px

#warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Подготовка данных

В данном разделе загрузим тестовый датасет и проведем с ним все те же манипуляции что и с тренировочным.

#### Загрузка датасета

In [2]:
#Прочитаем тестовый датасет

df1=pd.read_excel("datasets\\test_corpus.xlsx")

#### Приведение текстов к нижнему регистру

In [6]:
df1['TITLE']=df1['TITLE'].str.lower()

#### Замена англицизмов

In [7]:
%%time
df_engl=pd.read_csv("datasets\\Anglicisms(13-02-2022).csv",encoding = 'cp1251',sep=';')
df_engl
df_engl=df_engl.drop(['Unnamed: 0'], axis=1)
df_engl.columns=df_engl.columns.str.lower()
df_engl
df_engl['english']=df_engl['english'].str.lower()
df_engl.sample(10)

Wall time: 3.99 ms


,english,russian
61,politico,политико
10,rt de,рт доич
42,bayraktar,байрактар
27,spiegel,шпигель
4,sun,сан
62,the,
44,bild,билд
37,american,американ
77,covid,ковид
38,anadolu,анадолу


In [8]:
df_engl_dict=df_engl.set_index('english').T.to_dict('records')

<ipython-input-8-5cdb957739c6>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_engl_dict=df_engl.set_index('english').T.to_dict('records')


In [9]:
df_engl_dict=df_engl_dict[0]

In [10]:
def replace_from_dict(str,dict):
    for i in (str.split(" ")):
        for j in dict.keys():
            if i==j:
                #print(i)
                str=str.replace(i,dict.get(j))
    return str

In [11]:
#Очистим текст от лишних символов
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Zа-яА-я0-9 ]', ' ', text).split())  
clear_text('hill: сенат сша проголосует по вопросу санкций против «северного потока — 2» до 14 января')

'hill сенат сша проголосует по вопросу санкций против северного потока 2 до 14 января'

In [12]:
df1['TITLE']=df1['TITLE'].apply(lambda x: clear_text(x))
df1['TITLE']=df1['TITLE'].apply(lambda x: replace_from_dict(x,df_engl_dict))

#### Замена числительных

In [13]:
%%time
df_numbers=pd.read_csv("datasets\\Numbers(05-01-2022).csv",encoding = 'cp1251',sep=';')
df_numbers=df_numbers.drop(['Unnamed: 0'], axis=1)
df_numbers=df_numbers.dropna()
df_numbers.sample(10)


Wall time: 9.97 ms


,number,word
212,2003,две тысячи три
103,104,сто четыре
113,114,сто четырнадцать
100,101,сто один
139,140,сто сорок
233,2024,две тысячи двадцать четыре
142,143,сто сорок три
183,1400,тысяча четыреста
167,600,шестьсот
190,1941,тысяча девятьсот сорок один


In [14]:
df_numbers=df_numbers.set_index('number').T.to_dict('records')
df_numbers_dict=df_numbers[0]

<ipython-input-14-4f619f413480>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_numbers=df_numbers.set_index('number').T.to_dict('records')


In [15]:
df1['TITLE']=df1['TITLE'].apply(lambda x: replace_from_dict(x,df_numbers_dict))

#### Лемматизация

In [16]:
#Напишем фукнцию лемматизации
def lemmatization(texts):
    nlp = spacy.load("ru_core_news_sm")
    texts_out = []
    for text in tqdm(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [17]:
%%time
lemmatized_texts=lemmatization(df1['TITLE'])
#Проверим результат
print(lemmatized_texts[0:10])

100%|███████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 132.23it/s]

['президент россия и сша путин и байден более час обсуждать по телефон украина', 'роскосмос показать астероид приближаться к земля', 'представитель мид россия захаров заявить что украина нашпиговать натовский оружие', 'челси в эпоха абрамович выиграть весь турнир в которых принимать участие', 'захаров заявить о нашпигованности украина оружие нато', 'ушаков многие мировой лидер запросить телефонный переговоры с путин из за украина', 'в территориальный вода рф на курилах была обнаружить подводный лодка вмс сша', 'глава днр не исключить связь взрыв с провокация', 'челси победить палмейрас и впервые выиграть клубный чемпионат мир', 'зеленский обсудить с трюдо вопрос о деэскалация']
Wall time: 2.81 s


#### Подготовка корпуса

In [18]:
corpus=np.array(lemmatized_texts)
print("Длина корпуса: ",len(corpus))

Длина корпуса:  230


In [19]:
stop_words = set(stopwords.words('russian')) 
custom_stopwords = set(['это','весь','яя','ая','га','свой','спрашивать','становиться','отвечать','подходить','уходить','выходить','оставаться','разговор','ехать','приходить'])
stop_words.update(custom_stopwords)
additional_stopwords= set(['двадцать','глава','тысяча','год'])
#additional_stopwords= set(['двадцать','глава','тысяча','год','пять','девятнадцать'])
stop_words.update(additional_stopwords)


In [20]:
with open("dictionary.pkl", "rb") as f:
    dictionary_train = pickle.load(f)
print(len(dictionary_train))

11576


In [21]:
#Удалим стоп-слова
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

tokenized_texts = [list(tokenize(text.lower()))for text in corpus]
tokenized_texts_processed= remove_stopwords(tokenized_texts)
#dictionary2= Dictionary(tokenized_texts_processed)
#dictionary_train.add_documents(tokenized_texts_processed)

corpus_test= [dictionary_train.doc2bow(text) for text in tokenized_texts_processed]
#corpus
print(len(dictionary_train))

11576


Возможно, здесь стоит использовать HashDictionary и hashing trick, так как число слов стало больше. Но опустим этот момент на данном этапе моих знаний и обучения это видится сложновыполнимым и дальнейшие предсказания выглядят валидными.

# Тематическое моделирвование

#### LDA

In [24]:
#Загрузим предобученную модель
with open("model_lda_multi_5_topics.pkl", "rb") as f:
    lda_model_multi = pickle.load(f)

In [25]:
%%time
#Обновим модель на новом корпусе

lda_model_multi.update(corpus_test)

Wall time: 7.49 s


In [26]:
%%time
vector=lda_model_multi[corpus_test[0]]

Wall time: 0 ns


In [27]:
#Напечатаем топики из заготовленной модели
lda_model_multi.print_topics()

[(0,
  '0.032*"россия" + 0.025*"украина" + 0.025*"путин" + 0.023*"сша" + 0.022*"заявить" + 0.017*"президент" + 0.014*"нато" + 0.012*"рада" + 0.012*"песков" + 0.011*"днр"'),
 (1,
  '0.027*"ковид" + 0.026*"девятнадцать" + 0.018*"коронавирус" + 0.017*"россия" + 0.012*"омикрон" + 0.011*"сутки" + 0.011*"заявить" + 0.011*"мочь" + 0.011*"врач" + 0.009*"случай"'),
 (2,
  '0.067*"украина" + 0.046*"россия" + 0.037*"сша" + 0.017*"заявить" + 0.016*"рф" + 0.013*"подлодка" + 0.013*"запрет" + 0.012*"вторжение" + 0.012*"призвать" + 0.012*"байден"'),
 (3,
  '0.026*"олимпиада" + 0.023*"россия" + 0.019*"пекин" + 0.018*"выиграть" + 0.015*"территориальный" + 0.014*"вмс" + 0.012*"северный" + 0.011*"поток" + 0.011*"российский" + 0.010*"гонка"'),
 (4,
  '0.024*"россиянин" + 0.014*"алма" + 0.014*"казахстан" + 0.013*"сын" + 0.013*"политолог" + 0.012*"мвд" + 0.011*"память" + 0.011*"завоевать" + 0.010*"задержать" + 0.010*"мужчина"')]

#Топик 0 - политика внешняя, украинский вопрос<br>
#Топик 1 - коронавирус, внутренняя политика<br>
#Топик 2 - политика внешняя. Близко по смыслу к Топику 0 на Intertopic Distance Map<br>
#Топик 3 - олимпиада, спорт<br>
#Топик 4 - Казахстан, внутренняя/внешняя политика<br>

In [28]:
#Напишем функцию для предсказания наиболее вероятного топика
def topic_prediction(corpus_for_input):
    topics = sorted(list(lda_model_multi[corpus_for_input])[0],key=lambda x:x[1],reverse=True)
    return topics[0][0]

Проверим правильность предсказаний для самых очевидных топиков. <br>
Топик 1 - коронавирус

In [29]:
#Проверим правильность предсказаний для
for i in range(0,len(corpus)):
    if (topic_prediction(corpus_test[i]))==1:
        print(corpus[i])
 
        

смольный петербург преодолеть пик пятый волна коронавирус
власть петербург заявить о преодоление пик пятый волна коронавирус
в комитет по здравоохранение петербург заявить о преодоление пик пятый волна ковид девятнадцать
британский пользователь твиттер восхититься картина на фото шойгу и уоллес
эксперт заражение омикрон мочь привести к нейроковиду и системный воспаление
врач исаев омикрон мочь привести к нейроковиду и системный воспаление
в йемен похитить шесть сотрудник оон
врач комаровский объяснить что не каждый головной боль являться мигрень
врач комаровский назвать бесполезный лекарство в вид таблетка при сильный головной боль
врач евгений комаровский заявить что не каждый головной боль являться мигрень
в центр днком предупредить о вызвать ковид девятнадцать системный воспаление у ребёнок
врач исаев сообщить что коронавирус мочь привести к тяжёлый поражение нервный система
иммунолог ярцев посоветовать всегда иметь при себя экспресс тест на ковид девятнадцать
риа новость весь желат

**Похоже, но не везде.**<br>
Топик 3 - олимпиада, спорт

In [30]:
#Проверим правильность предсказаний для
for i in range(0,len(corpus)):
    if (topic_prediction(corpus_test[i]))==3:
        print(corpus[i])
        

челси в эпоха абрамович выиграть весь турнир в которых принимать участие
челси победить палмейрас и впервые выиграть клубный чемпионат мир
челси выиграть весь возможный трофей после покупка клуб абрамович
челси выиграть двадцать один й трофей при абрамовиче
челси впервые стать победитель клубный чемпионат мир
россия уступить чехия в матч групповой этап олимпиада
челси обыграть палмейрас и выиграть клубный чемпионат мир
нок украина выступить с масштабный обращение с призыв к мир во время ои два тысяча двадцать два
фигуристка валиев получить двадцать шесть й стартовый номер в короткий программа на олимпиада
валиева выступить двадцать шесть й в короткий программа на олимпиада в пекин
российский кёрлингист обыграть италия на олимпиада два тысяча двадцать два в пекин
адесанья победить уиттакера и в четвёртый раз защитить титул ufc в средний вес
большунов упрочить преимущество сборная россия в эстафета на ои два тысяча двадцать два до минута
мужской сборная россия впервые в новый история выи

**По тематике олимпиады и спорта пресдказания достаточно точные, но есть ошибки то же**

# Выводы

1.Для дальнейшего улучшения тематического моделирования  стоит улучшить предобработку. Стоит работать с числительными, но осторожно: <br>
Модель видимо ориентируется на них для предскзания тем по коронавирусу и спорту.<br>
2.Стоит изучать инстурменты для урвановешевания классов в датасете - у нас большинство новостей про внешнюю политику.<br>
3.Новости о внешней политике, хоть и делятся при помощи LDA на несколько топиков  - для человека слабоотличимы.<br>
4.Для обучения, апдейта и инференса мы дожны использовать один и тот же словарь, что не совсем правильно. Верней было бы исполльзовать HashDictionary и hashing trick, что на данном этапе моих знаний видится сложно выполнимым. Возможно эти инстурменты уже используются "под капотом" LDA
